In [5]:
#import the necessary modules
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from numpy  import array
from random import randrange
from imutils import paths
import timeit
#load the image
#img_original = cv2.imread('/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid1HQ.jpg')
#plt.imshow(img_original)
#function to convert the cell (string) to list of coordinates (x,y) in order to
#be easier to extract the coordinates later
def Convert(string): 
    li = list(string.split(",")) 
    return li 

In [6]:
nests_folder="/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images"
imagePathsnests = sorted(list(paths.list_images(nests_folder)))
imagePathsnests

['/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid3MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5MQ.jpg']

In [7]:
#imagePathsnests=imagePathsnests[2:10]

In [8]:
imagePathsnests

['/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid3MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5MQ.jpg']

In [9]:
#opencv opens the image with the wrong colour channels
#this code corrects that
start = timeit.default_timer()
for image_path in imagePathsnests:
    img_original=cv2.imread(image_path)
    img=cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
    #loop
    #load of the manually annotated polygons using VGG annotator
    df_original= pd.read_csv("/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/FiveNestAnnotedTest2.csv")
    #select only the current image
    df=df_original[df_original["filename"]==image_path.split("/")[-1:][0]]
    #print dataframe
    #print(df)
    #loop through all lines of the df to get the coordinates of all polygons for same image
    #create a list to store all xx and all yy of all polygons
    all_coordx=[]
    all_coordy=[]
    #reset the index of the line so that the next loop doesn't give error
    df=df.reset_index()
    for d in range(0, len(df)):
    #remove the text that is not needed and keep only the coordinates
        coord=df["region_shape_attributes"][d].replace('{"name":"polygon","all_points_x":', "")
        coord=coord.replace(',"all_points_y":', "")
        coord=coord.replace(']', "")
        coord=coord.replace('}', "")
        #create a list for the xx coordinates and another for the yy coordinates
        coordx=coord.split("[")[1]
        coordx=Convert(coordx)
        coordy=coord.split("[")[2]
        coordy=Convert(coordy)
        all_coordx.append(coordx)
        all_coordy.append(coordy)
    window=10
    square_index=1
    #double loop to go through all polygons and within each polygon create a list with all points
    for p in range(0,len(all_coordx)):
        #create a final list by merging the xx and yy lists
        polycoordinates=[]
        coordx=all_coordx[p]
        coordy=all_coordy[p]
        for i in range(len(coordx)):
            polycoordinates.append([coordx[i],coordy[i]])
            #convert to array in order to extract the background later
        polycoordinates = array(polycoordinates, np.int32)
        #Crop the bounding box of the polygon
        rect = cv2.boundingRect(polycoordinates)
        x,y,w,h = rect
        croped = img_original[y:y+h, x:x+w].copy()
        # create a mask corresponding to the area of the selected polygon
        polycoordinates = polycoordinates - polycoordinates.min(axis=0)
        mask = np.zeros(croped.shape[:2], np.uint8)
        cv2.drawContours(mask, [polycoordinates], -1, (255, 255, 255), -1, cv2.LINE_AA)
        #remove the background
        dst = cv2.bitwise_and(croped, croped, mask=mask)
        #re-convert the polygon array to list again in order to
        #use the functions from the shapely module. this module includes a function
        #that allows to check if a point is inside a polygon, using functions "Point(x,y)"
        #and "Polygon(list_of_coordinates)"
        polygon_x=[]
        for j in range(0,len(polycoordinates)):
            polygon_x.append((polycoordinates[j][0],polycoordinates[j][1]))
        #convert to polygon object type of shapely module
        poly_final=Polygon(polygon_x)
        #this loop makes random squares and
        #keeps the ones that are inside the region of finterest
        #create a copy of the image for ilustration
        dst_test=dst.copy()
        

        #get maximum length
        max_size=max(dst.shape)
        r=[0.2,0.25,0.3,0.35,0.4,0.45,0.5]
        for ri in r:
            size=max_size*ri
            #loop until it reaches the number of squares needed
            #create a variable to save the yy point
            #current_yy=0
            #number of squares needed
            attempts=0
            #create a list to store the squares
            squares=[]
            num_squares=50
            print(ri)

            #loop until it reaches the number of squares needed
            while len(squares)<num_squares:
                if attempts>2000:
                    break
                print(attempts)
                #randomly chose a x and a y coordinates
                current_xx=randrange(0, dst.shape[0])
                current_yy=randrange(0, dst.shape[1])
                #randomly chose the length of the edges of the square (in this case between 10 to 50)
                #get all points inside the randomly determined square
                X, Y = np.mgrid[current_xx:current_xx+size, current_yy:current_yy+size]
                points=np.vstack((X.ravel(), Y.ravel()))
                #variable to note if all points are inside the polygon of interest
                all_true=1
                attempts=attempts+1
                #loop through all points and if any of the points is outside of the 
                #region of interest change "all_true" variable to 0
                for ii in range(0, len(points[0])):
                    if poly_final.contains((Point(points[0][ii], points[1][ii]))) == False:
                        all_true=0
                        break
                #only if all_true=1 include that square in the list
                if all_true==1:
                    squares.append([(round(current_xx),round(current_yy)),(round(current_xx+size),round(current_yy+size))])
                    cropped=dst_test[round(current_yy):round(current_yy)+round(size),round(current_xx):round(current_xx)+round(size)]
                    #export cropped image to a folder
                    print(len(squares))
                    filename="/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/"
                    filename=filename+image_path.split("/")[-1:][0].split(".")[0]+"_"+"Polygon"+str(p+1)+"_"+str(ri)+"_"+str(square_index)+".jpg"
                    print(filename)
                    square_index=square_index+1
                    cv2.imwrite(filename, cropped)
            finaltest=dst.copy()
            for i in range(0, len(squares)):
                finaltest=cv2.rectangle(finaltest,squares[i][0], squares[i][1],(0,255,0),3)
            filename_final="/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/"
            filename_final=filename_final+image_path.split("/")[-1:][0].split(".")[0]+"_"+"Polygon"+str(p+1)+"_"+str(ri)+"_"+"Final"+".jpg"
            cv2.imwrite(filename_final, finaltest)
stop = timeit.default_timer()

print('Time: ', stop - start)

0.2
0
1
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_1.jpg
1
2
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_2.jpg
2
3
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_3.jpg
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
4
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_4.jpg
25
26
27
5
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_5.jpg
28
29
30
31
32
33
34
35
36
37
38
6
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_6.jpg
39
40
41
7
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_7.jpg
42
8
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_8.jpg
43
44
9
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_9.jpg
45
46
47
48
10
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_10.jpg
49
50
51
52
53
11
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.2_11.jpg
54
55
56
57
58
59
60
61
62
63
64
65
12
/mnt/ubun

304
305
306
307
308
309
37
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_87.jpg
310
311
312
313
314
315
38
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_88.jpg
316
39
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_89.jpg
317
318
40
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_90.jpg
319
320
41
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_91.jpg
321
322
323
42
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_92.jpg
324
325
326
327
43
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_93.jpg
328
44
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_94.jpg
329
45
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_95.jpg
330
46
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_96.jpg
331
332
333
334
335
47
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.25_97.jpg
336
337
338
339
48
/mnt/ubuntu_disk/Ubuntu_driv

46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
2
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.35_152.jpg
67
68
69
70
71
72
73
74
3
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.35_153.jpg
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
4
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.35_154.jpg
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
5
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.35_155.jpg
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
6
/mnt/ubuntu_disk/Ubuntu_

1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
43
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid3MQ_Polygon1_0.35_193.jpg
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454

1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465


...

122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
7
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_307.jpg
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
8
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_308.jpg
177
178
179
180
181
182
9
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_309.jpg
183
184
185
186
187
188
189
190
191
192
193
10
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_310.jpg
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
11
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_311.jpg
209
210
211
212
213
214
215
216
217
218
219
220
221
12
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_312.jpg
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
13
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5HQ_Polygon1_0.5_313.jpg
238
239
240
24

67
27
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_27.jpg
68
69
70
71
28
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_28.jpg
72
73
74
75
76
29
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_29.jpg
77
30
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_30.jpg
78
31
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_31.jpg
79
80
81
82
83
84
32
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_32.jpg
85
86
87
88
33
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_33.jpg
89
34
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_34.jpg
90
91
92
93
94
95
35
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_35.jpg
96
97
98
99
36
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_36.jpg
100
37
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_37.jpg
101
38
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.2_38.jpg
102
3

25
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_125.jpg
105
106
107
26
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_126.jpg
108
109
27
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_127.jpg
110
28
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_128.jpg
111
112
113
114
115
29
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_129.jpg
116
117
30
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_130.jpg
118
119
120
121
122
123
124
125
126
127
128
129
130
31
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_131.jpg
131
132
32
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_132.jpg
133
134
135
136
137
138
139
140
33
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_133.jpg
141
142
143
144
145
146
147
148
149
150
34
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.3_134.jpg
151
152
35
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Poly

76
77
78
18
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_218.jpg
79
80
81
82
83
84
85
86
87
19
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_219.jpg
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
20
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_220.jpg
107
108
109
110
111
112
21
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_221.jpg
113
114
115
116
22
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_222.jpg
117
23
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_223.jpg
118
119
24
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_224.jpg
120
25
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_225.jpg
121
26
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_226.jpg
122
123
124
125
126
127
128
129
130
27
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.4_227.jpg
131
132
133
134
135
136
137
138
139
140
141
28
/mnt/ub

383
384
385
386
387
388
389
390
391
392
393
394
395
396
46
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.45_296.jpg
397
398
399
400
401
402
403
404
405
47
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.45_297.jpg
406
407
408
409
410
48
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.45_298.jpg
411
412
413
414
415
416
417
418
419
420
49
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.45_299.jpg
421
422
423
424
425
426
427
428
50
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.45_300.jpg
0.5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.5_301.jpg
15
16
17
18
19
20
21
22
2
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.5_302.jpg
23
24
25
26
27
28
29
30
31
3
/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/Nid5MQ_Polygon1_0.5_303.jpg
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
7